In [10]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from math import cos, sin
from math import sqrt

In [11]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Homogeneous transformation

In [17]:
def homogeneous(rot_x, rot_y, rot_z, trans_x, trans_y, trans_z):
    R_x = np.array([[1, 0, 0, 0],
                    [0, cos(rot_x), -sin(rot_x), 0],
                    [0, sin(rot_x), cos(rot_x), 0],
                    [0, 0, 0, 1]])
    R_y = np.array([[cos(rot_y), 0, sin(rot_y), 0],
                    [0, 1, 0, 0],
                    [-sin(rot_y), 0, cos(rot_y), 0],
                    [0, 0, 0, 1]])
    R_z = np.array([[cos(rot_z), -sin(rot_z), 0, 0],
                    [sin(rot_z), cos(rot_z), 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
    T = np.array([[1,0,0,trans_x],
                   [0,1,0,trans_y],
                   [0,0,1,trans_z],
                   [0,0,0,1]])
    
    T_t = R_x @ R_y @ R_z
    return T_t
    

### Roll Pitch Yaw

In [21]:
def rpy(z, y, x):
    p0 = np.array([0, 0, 0, 1]).flatten()
    p0_x = np.array([5,0,0,1]).flatten()
    p0_y = np.array([0,5,0,1]).flatten()
    p0_z = np.array([0,0,5,1]).flatten()

    p1_r_z = homogeneous(0,0,z,0,0,0)
    p1_r_y = homogeneous(0,y,0,0,0,0)
    p1_r_x = homogeneous(x,0,0,0,0,0)
    
    p1 = p1_r_z @ p1_r_y @ p1_r_x @ p0
    p1_x = p1_r_z @ p1_r_y @ p1_r_x @ p0_x
    p1_y = p1_r_z @ p1_r_y @ p1_r_x @ p0_y
    p1_z = p1_r_z @ p1_r_y @ p1_r_x @ p0_z

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_zlim(0, 10);
    
    ax.scatter3D(p1[0], p1[1], p1[2])
    ax.plot3D([p1[0], p1_x[0]], [p1[1], p1_x[1]], [p1[2], p1_x[2]], 'r')
    ax.plot3D([p1[0], p1_y[0]], [p1[1], p1_y[1]], [p1[2], p1_y[2]], 'g')
    ax.plot3D([p1[0], p1_z[0]], [p1[1], p1_z[1]], [p1[2], p1_z[2]], 'b')
    
interactive_plot = interactive(rpy, x=(-pi, pi, pi/16), y=(-pi, pi, pi/16), z=(-pi, pi, pi/16))
output = interactive_plot.children[-1]
interactive_plot


interactive(children=(FloatSlider(value=0.0, description='z', max=3.141592653589793, min=-3.141592653589793, s…

### Euler Angle

In [23]:
def euler(alpha, beta, gamma):
    p0 = np.array([0, 0, 0, 1]).flatten()
    p0_x = np.array([5,0,0,1]).flatten()
    p0_y = np.array([0,5,0,1]).flatten()
    p0_z = np.array([0,0,5,1]).flatten()

    p1_r_z_alpha = homogeneous(0,0,alpha,0,0,0)
    p1_r_y_beta = homogeneous(0,beta,0,0,0,0)
    p1_r_z_gamma = homogeneous(0,0,gamma,0,0,0)
    
    T = p1_r_z_alpha @ p1_r_y_beta @ p1_r_z_gamma
    
    p1 = T @ p0
    p1_x = T @ p0_x
    p1_y = T @ p0_y
    p1_z = T @ p0_z

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_zlim(0, 10);
    
    ax.scatter3D(p1[0], p1[1], p1[2])
    ax.plot3D([p1[0], p1_x[0]], [p1[1], p1_x[1]], [p1[2], p1_x[2]], 'r')
    ax.plot3D([p1[0], p1_y[0]], [p1[1], p1_y[1]], [p1[2], p1_y[2]], 'g')
    ax.plot3D([p1[0], p1_z[0]], [p1[1], p1_z[1]], [p1[2], p1_z[2]], 'b')
    
interactive_plot = interactive(euler, alpha=(-pi, pi, pi/16), beta=(-pi, pi, pi/16), gamma=(-pi, pi, pi/16))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='alpha', max=3.141592653589793, min=-3.14159265358979…

In [88]:
import quaternion

def quaternion_f(q0, q1, q2, q3):
    p = np.array([0, 0, 0, 0]).flatten()
    p_x = np.array([0,2,0,0]).flatten()
    p_y = np.array([0,0,2,0]).flatten()
    p_z = np.array([0,0,0,2]).flatten()


    q = np.array([q0,q1,q2,q3])

    q_inv = np.conj(q)

    p1 = (q[0]**2  - np.dot(q[1:], q[1:]))*p[1:] + 2*q[0]*(np.cross(q[1:], p[1:])) + 2*q[1:]*(np.dot(q[1:], p[1:]))
    p1_x = (q[0]**2  - np.dot(q[1:], q[1:]))*p_x[1:] + 2*q[0]*(np.cross(q[1:], p_x[1:])) + 2*q[1:]*(np.dot(q[1:], p_x[1:]))
    p1_y = (q[0]**2  - np.dot(q[1:], q[1:]))*p_y[1:] + 2*q[0]*(np.cross(q[1:], p_y[1:])) + 2*q[1:]*(np.dot(q[1:], p_y[1:]))
    p1_z = (q[0]**2  - np.dot(q[1:], q[1:]))*p_z[1:] + 2*q[0]*(np.cross(q[1:], p_z[1:])) + 2*q[1:]*(np.dot(q[1:], p_z[1:]))

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_zlim(0, 10);
    
    ax.scatter3D(p1[0], p1[1], p1[2])
    ax.plot3D([p1[0], p1_x[0]], [p1[1], p1_x[1]], [p1[2], p1_x[2]], 'r')
    ax.plot3D([p1[0], p1_y[0]], [p1[1], p1_y[1]], [p1[2], p1_y[2]], 'g')
    ax.plot3D([p1[0], p1_z[0]], [p1[1], p1_z[1]], [p1[2], p1_z[2]], 'b')

    
interactive_plot = interactive(quaternion_f, q0=(-1.5, 1.5, 0.1), q1=(-1.5, 1.5, 0.1), q2=(-1.5, 1.5, 0.1), q3=(-1.5, 1.5, 0.1))
output = interactive_plot.children[-1]
interactive_plot
    

interactive(children=(FloatSlider(value=0.0, description='q0', max=1.5, min=-1.5), FloatSlider(value=0.0, desc…

In [108]:
def quaternion_easy_func(q0, q1, q2, q3):
    q = np.quaternion(q0, q1, q2, q3)
    p_x = np.quaternion(0,1,0,0)
    p_y = np.quaternion(0,0,1,0)
    p_z = np.quaternion(0,0,0,1)
    p = np.quaternion(0,0,0,0)

    p1 = quaternion.as_float_array(q*p)[1:]
    p1_x = quaternion.as_float_array(q*p_x)[1:]
    p1_y = quaternion.as_float_array(q*p_y)[1:]
    p1_z = quaternion.as_float_array(q*p_z)[1:]
    
    
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.set_zlim(0, 10);
    
    ax.scatter3D(p1[0], p1[1], p1[2])
    ax.plot3D([p1[0], p1_x[0]], [p1[1], p1_x[1]], [p1[2], p1_x[2]], 'r')
    ax.plot3D([p1[0], p1_y[0]], [p1[1], p1_y[1]], [p1[2], p1_y[2]], 'g')
    ax.plot3D([p1[0], p1_z[0]], [p1[1], p1_z[1]], [p1[2], p1_z[2]], 'b')

interactive_plot = interactive(quaternion_easy_func, q0=(-1.5, 1.5, 0.1), q1=(-1.5, 1.5, 0.1), q2=(-1.5, 1.5, 0.1), q3=(-1.5, 1.5, 0.1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='q0', max=1.5, min=-1.5), FloatSlider(value=0.0, desc…